In [21]:
import numpy as np
import pyuvdata
import hera_cal as hc
import uvtools
import random
import threading
import glob

In [ ]:
def load_relevant_data(miriad_path, calfits_path):
    """TODO: docstring"""

    uvd = pyuvdata.UVData()
    uvd.read_miriad(miriad_path)

    # get the redundancies for that data
    aa = hc.utils.get_aa_from_uv(uvd)
    info = hc.omni.aa_to_info(aa)
    red_bls = np.array(info.get_reds())

    # gains for same data 
    gains, flags = hc.io.load_cal(calfits_path)
    
    return red_bls, gains, uvd

In [4]:
def get_good_red_bls(red_bls, gain_keys, min_group_len = 4):
    """Select all the good antennas from red_bls
    
    Each baseline group in red_bls has its bad separations removed. Groups with less than min_group_len are removed.
    
    Arguments:
    
        red_bls : list of lists - Each sublist is a group of redundant separations for a unique baseline.
        gain_keys : dict - gains.keys() from hc.io.load_cal()
        min_group_len: int - Minimum number of separations in a 'good' sublist.
                            (Default = 4, so that both training and testing can take two seps)
    
    Returns:
    
        list of lists - Each sublist is a len >=4 list of separations of good antennas
    
    """
    
    def ants_good(sep):
        """Returns True if both antennas are good.

        Because we are using data from firstcal (is this right? I have trouble rememebring the names and properties of the different data sources)
        we can check for known good or bad antennas by looking to see if the antenna is represented in gains.keys(). If the antenna is present, its good.

        Arguments:

            sep : tuple - antenna indices

        Returns :

            bool - True if both antennas are in gain_keys
        """

        ants = [a[0] for a in gain_keys]

        if sep[0] in ants and sep[1] in ants:
            return True

        else:
            return False

    good_redundant_baselines = []
    
    for group in red_bls:
        
        new_group = []
        
        for sep in group:
            
            # only retain seps made from good antennas
            if ants_good(sep) == True:
                new_group.append(sep)
                
        new_group_len = len(new_group)
        
        # make sure groups are large enough that both the training set and the testing set can take two seps 
        if new_group_len >= min_group_len:
            
            # Make sure groups are made from even number of seps
            #
            # I honestly dont recall why I did this. ¯\_(ツ)_/¯ 
            if new_group_len % 2 != 0:
                new_group.pop()
                
            good_redundant_baselines.append(sorted(new_group))
            
    return good_redundant_baselines

In [5]:
def train_test_split_red_bls(red_bls, training_percent = 0.80):
    """Slit a list of redundant baselines into a training set and a testing set. 
    
    Each of the two sets has at least one pair of baselines from every group from red_bls. However, separations from one set will not appear the other.
    
    Arguments:
    
        red_bls : list of lists - Each sublist is a group of redundant separations for a unique baseline. Each sublist must have at least 4 separations.
        training_percent : float - *Approximate* portion of the separations that will appear in the training set.
        
    Returns:
    
        tuple of dicts - train_red_bls_dict, test_red_bls_dict
    
    """
    
    
    training_redundant_baselines_dict = {}
    testing_redundant_baselines_dict = {}

    # make sure that each set has at least 2 seps from each group
    #
    thinned_groups_dict = {}
    for group in red_bls:

        # group key is the sep with the lowest antenna indicies
        key = sorted(group)[0]
        
        # pop off seps from group and append them to train or test groups
        random.shuffle(group)
        training_group = []
        training_group.append(group.pop())
        training_group.append(group.pop())

        testing_group = []
        testing_group.append(group.pop())
        testing_group.append(group.pop())
        
        # add the new train & test groups into the dicts
        training_redundant_baselines_dict[key] = training_group
        testing_redundant_baselines_dict[key] = testing_group
        
        # if there are still more seps in the group, save them into a dict for later assignment
        if len(group) != 0:
            thinned_groups_dict[key] = group

    # Shuffle and split the group keys into two sets using 
    #
    thinned_dict_keys = thinned_groups_dict.keys()
    random.shuffle(thinned_dict_keys)
    
    """Because we are ensuring that each set has some seps from every group, the ratio of train / test gets reduced a few percent.
    This accounts for that with an arbitrary shift found by trial and error."""
    training_percent = np.min([0.95, training_percent + 0.15])

    
    # why did i call this extra?
    # these are the keys that each set will extract seps from thinned_groups_dict with
    training_redundant_baselines_extra, testing_redundant_baselines_extra = np.split(thinned_dict_keys, [int(len(thinned_dict_keys)*training_percent)])

    # extract seps from thinned_groups_dict and apply to same key in training set
    for key in training_redundant_baselines_extra:
        key = tuple(key)
        group = thinned_groups_dict[key]
        training_group = training_redundant_baselines_dict[key]
        training_group.extend(group)
        training_redundant_baselines_dict[key] = training_group

    # extract seps from thinned_groups_dict and apply to same key in testing set
    for key in testing_redundant_baselines_extra:
        key = tuple(key)
        group = thinned_groups_dict[key]
        testing_group = testing_redundant_baselines_dict[key]
        testing_group.extend(group)
        testing_redundant_baselines_dict[key] = testing_group
        
    return training_redundant_baselines_dict, testing_redundant_baselines_dict

In [6]:
def get_or_gen_test_train_red_bls_dicts(red_bls = None, gain_keys = None, training_percent = 0.80):

    if len(glob.glob("*.npz")) == 2:
        
        training_redundant_baselines_dict = loadnpz('training_redundant_baselines_dict.npz')[()]
        testing_redundant_baselines_dict = loadnpz('testing_redundant_baselines_dict.npz')[()]
    else:
        
        assert type(red_bls) != None, "Provide a list of redundant baselines"
        assert type(gain_keys) != None, "Provide a list of gain keys"
        
        good_redundant_baselines = get_good_red_bls(red_bls, gain_keys)
        training_redundant_baselines_dict, testing_redundant_baselines_dict = train_test_split_red_bls(good_redundant_baselines, training_percent = training_percent)

        np.savez('training_redundant_baselines_dict', training_redundant_baselines_dict)
        np.savez('testing_redundant_baselines_dict', testing_redundant_baselines_dict)

    return training_redundant_baselines_dict, training_redundant_baselines_dict

In [7]:
from numpy import load
def loadnpz(filename):
    """Loads up npzs. For dicts do loadnpz(fn)[()]"""
    a = load(filename)
    d = dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a)))
    return d['data1arr_0']

In [8]:
def get_seps_data(red_bls_dict, uvd):
    """Get the data and data.conjugate() for all the seps in a redundant baselines dictionary."""
    
    data = {}
    for key in red_bls_dict.keys():
        for sep in red_bls_dict[key]:
            data[sep] = uvd.get_data(sep)
    
    return data

In [9]:
class Data_Creator(object):
    """Creates data in an alternate thread.

    ## usage:
    ## data_maker = data_creator(num_flatnesses=250, mode = 'train')
    ## data_maker.gen_data() (before loop)
    ## inputs, targets = data_maker.get_data() (start of loop)
    ## data_maker.gen_data() (immediately after get_data())

    """
    
    """TODO:
    
    Comments, arg / return explanation & genreal cleanup
    
    """

    def __init__(self, num_flatnesses, bl_data = None, bl_dict = None, gains = None, clean = False):
        
        
        self._num = num_flatnesses
        self._clean = clean
                    
        self._bl_data = bl_data
        self._bl_data_c = None
        
        self._bl_dict = bl_dict
        
        self._gains = gains
        self._gains_c = None
        
        self._epoch_batch = []
        self._nu = np.arange(1024)
        
    def _gen_data(self):
        
        # scaling tools
        # the NN likes data in the range (0,1)
        angle_tx  = lambda x: (np.asarray(x) + np.pi) / (2. * np.pi)
        angle_itx = lambda x: np.asarray(x) * 2. * np.pi - np.pi

        abs_min_max_delay = 0.040
        delay_tx  = lambda x: (np.array(x) + abs_min_max_delay) / (2. * abs_min_max_delay)
        delay_itx = lambda x: np.array(x) * 2. * abs_min_max_delay - abs_min_max_delay

        if self._clean == True:
            
            targets = np.random.uniform(low = -.040, high = 0.040, size = (self._num * 60, 1))
            inputs = np.angle(np.exp(-2j * np.pi * (targets * self._nu + np.random.uniform())))
            
            self._epoch_batch.append((angle_tx(inputs), delay_tx(targets)))
            
        else:
            assert type(self._bl_data) != None, "Provide visibility data"
            assert type(self._bl_dict) != None, "Provide dict of baselines"
            assert type(self._gains)   != None, "Provide antenna gains"
            
            if self._bl_data_c == None:
                self._bl_data_c = {key : self._bl_data[key].conjugate() for key in self._bl_data.keys()}
                
            if self._gains_c == None:
                self._gains_c = {key : self._gains[key].conjugate() for key in self._gains.keys()}
                

            def _flatness(seps):
                """Create a flatness from a given pair of seperations, their data & their gains."""

                a, b = seps[0][0], seps[0][1]
                c, d = seps[1][0], seps[1][1]

                return self._bl_data[seps[0]]*self._bl_data_c[seps[1]] * self._gains_c[(a,'x')] * self._gains[(b,'x')] * self._gains[(c,'x')] * self._gains_c[(d,'x')]

            inputs = []
            for _ in range(self._num):

                unique_baseline = random.sample(self._bl_dict.keys(), 1)[0]
                two_seps = [random.sample(self._bl_dict[unique_baseline], 2)][0]
                
                inputs.append(_flatness(two_seps))

            targets = np.random.uniform(low = -.040, high = 0.040, size = (self._num * 60, 1))
            inputs = np.angle(np.array(inputs).reshape(-1,1024) * np.exp(-2j * np.pi * (targets * self._nu + np.random.uniform())))

            self._epoch_batch.append((angle_tx(inputs), delay_tx(targets)))

    def gen_data(self):
        
        self._thread = threading.Thread(target = self._gen_data, args=())
        self._thread.start()

    def get_data(self, timeout = 10):
        
        if len(self._epoch_batch) == 0:
            self._thread.join(timeout)
            
        return self._epoch_batch.pop(0)